In [12]:
import os
from astropy.table import Table,vstack,hstack,join
import astropy
import numpy as np

In [13]:
# Get the galaxy's central RA and DEC fron NED
basedir = '/home/jovyan/carmaedge/edge-sql-base/'
db1 = Table.read(basedir+'global_values/external/edge_ned.csv', format='ascii.ecsv')
db2 = Table.read(basedir+'global_values/derived/edge_rfpars.csv', format='ascii.ecsv')
db = join(db1, db2, keys='Name')

In [19]:
runs=['gal_nrad8','gal_nrad8_smolist','gal_nrad8_vdisp8','gal_nrad8_vdisp8_smolist']
with open('detected.txt') as f:
    namelist = f.read().splitlines()
gallist = [gal for gal in namelist if not gal.startswith("#")]
with open('gallist_fitctr.txt') as f:
    namelist_fitctr = f.read().splitlines()
gallist_fitctr = [gal for gal in namelist_fitctr if not gal.startswith("#")]
gaufit_table=Table.read('mergeTable/ned_offset_compare.txt',format='ascii.ecsv')

['galname',
 'ractr_ned',
 'dcctr_ned',
 'ractr_fit',
 'dcctr_fit',
 'offset',
 'Unc_SemiMajor',
 'Unc_SemiMinor']

In [35]:
ringlogs=['ringlog1','ringlog2']
for run in runs:
    tablelist_run=[]
    for gal in gallist:
        tablelist_rings=[]
        for ringlog in ringlogs:
            ## Read ringlog1 also - it will have 1's for PAFIT and INCFIT (and others in FREE), but ringlog2 will have -1 (except VROT and VDISP).
            #
            #
            if not os.path.isfile(run+'/output/'+gal[:8]+'/'+ringlog+'.txt'):
                    continue
            try:
                gal_ringlog = Table.read(run+'/output/'+gal[:8]+'/'+ringlog+'.txt',format='ascii')
            except:
                print('reading ringlog2 for '+gal+' failed, probably because the table is empty')
                continue
            gal_ringlog.remove_columns(['RAD(Kpc)', 'Z0(pc)', 'SIG(E20)', 'VRAD(km/s)'])


            #add columns:GAL_NAME, RACTR (deg), DCCTR (deg), INCFIT (0/1), PAFIT (0/1), XPOSFIT (0/1), YPOSFIT (0/1), VSYSFIT (0/1),VDISPFIT (0/1), FTYPE, WFUNC, BADFLAG
            #for simplicity, create a new table first
            row_number = gal_ringlog['RAD(arcs)'].shape[:][0]
            t_addcol=Table()
            GAL_NAME = [gal]*row_number
            if gal in gallist_fitctr: #check whether the ctrpos used is the 2dgaufitted one
                ractr_deg = gaufit_table[np.argwhere(gaufit_table['galname']==gal)[0][0]]['ractr_fit']
                dcctr_deg = gaufit_table[np.argwhere(gaufit_table['galname']==gal)[0][0]]['dcctr_fit']
                offset = row_number*[gaufit_table[np.argwhere(gaufit_table['galname']==gal)[0][0]]['offset']]
            else:
                i = np.where(db['Name'] == gal)[0][0]
                ractr_deg = [db['nedRA'][i]]*row_number
                dcctr_deg = [db['nedDE'][i]]*row_number
                offset = row_number*[0]

            parfile_path = run+'/param_'+gal+'.par'
            with open(parfile_path) as parfile:
                parfile_lines = parfile.read().splitlines()
            freepars = [line for line in parfile_lines if line.startswith('FREE')][0]
            freepars = freepars.split()[1:]
            #freepar is a list containing the free parameters

            #create a dictionary containing all the parameters
            param_dic={'VDISP':[0]*row_number,'PA':[0]*row_number,'VSYS':[0]*row_number,
                       'INC':[0]*row_number,'XPOS':[0]*row_number,'YPOS':[0]*row_number,'VROT':[0]*row_number}
            #check which is free
            exception = ['VROT','VDISP']
            for freepar in freepars:
                if ringlog == 'ringlog2':
                    if freepar not in exception:
                        param_dic[freepar]=[-1]*row_number
                    else:
                        param_dic[freepar]=[1]*row_number
                else:
                    param_dic[freepar]=[1]*row_number

            #extract FTYPE and WFUNC
            FTYPE = [line for line in parfile_lines if line.startswith('FTYPE')][0].split()[1]
            WFUNC = [line for line in parfile_lines if line.startswith('WFUNC')][0].split()[1]
            FTYPE = [int(FTYPE)]*row_number
            WFUNC = [int(WFUNC)]*row_number

            #set BADFLAG to 1 if any plot is missing
            BADFLAG = [0]*row_number
            plot_paths=['/plot_pv_local.pdf','/plot_parameters.pdf','/plot_maps_local.pdf','/plot_chanmaps_local.pdf']
            for plot_path in plot_paths:
                #gal[:8] is because the rest characters are truncated.
                if not os.path.isfile(run+'/output/'+gal[:8]+plot_path): 
                    BADFLAG = [1]*row_number
                    break

            t_addcol['GAL_NAME']=GAL_NAME
            t_addcol['RACTR']=ractr_deg
            t_addcol['DCCTR']=dcctr_deg
            t_addcol['offset'] = offset
            t_addcol['XPOSFIT(-1/0/1)']=param_dic['XPOS']
            t_addcol['YPOSFIT(-1/0/1)']=param_dic['YPOS']
            t_addcol['INCFIT(-1/0/1)']=param_dic['INC']
            t_addcol['PAFIT(-1/0/1)']=param_dic['PA']
            t_addcol['VSYSFIT(-1/0/1)']=param_dic['VSYS']
            t_addcol['VDISPFIT(0/1)']=param_dic['VDISP']
            t_addcol['VROTFIT(0/1)']=param_dic['VROT']
            t_addcol['FTYPE']=FTYPE
            t_addcol['WFUNC']=WFUNC
            t_addcol['BADFLAG']=BADFLAG
            
    #        print(t_addcol)
            t_galring = hstack([t_addcol,gal_ringlog])
            tablelist_rings.append(t_galring)
        #some gals only have ringlog1, or no ringlog at all since it is not contained in that run
        if len(tablelist_rings)==0:
            continue
        elif len(tablelist_rings)==1:
            t_gal=tablelist_rings[0]
        else:
            t_gal=vstack(tablelist_rings)
        tablelist_run.append(t_gal)
#        print(t_addcol)
    t_run = vstack(tablelist_run)
    
    renamecols = ['RAD(arcs)', 'VROT(km/s)', 'DISP(km/s)','INC(deg)','P.A.(deg)','Z0(arcs)','XPOS(pix)','YPOS(pix)','VSYS(km/s)']
    for colname in renamecols:
        t_run.rename_column(colname, colname[:colname.find('(')])
    print (t_run.colnames)
    fitparam = ['XPOSFIT(-1/0/1)','YPOSFIT(-1/0/1)','INCFIT(-1/0/1)','PAFIT(-1/0/1)','VSYSFIT(-1/0/1)','VDISPFIT(0/1)','VROTFIT(0/1)']
    Kmsparam = ['VROT','DISP','VSYS','E_VROT1','E_VROT2','E_DISP1','E_DISP2','E_VSYS1','E_VSYS2']
    Degparam = ['RACTR','DCCTR','INC','P.A.','E_PA1','E_PA2']
    Arcsparam = ['RAD','Z0','offset']
    Pixparam = ['XPOS','YPOS']
    
    t_run['BADFLAG'].description = 'BADFLAG = 1 when some ringlog exits, but Bbarolo fails to produce at least one of the plots'
    for col in fitparam:
        t_run[col].description = '0=fixed to input value; 1=fitted per ring; -1=fixed to mean value of previous fit'
    for col in Kmsparam:
        if ('vdisp' in run) and ('E_DISP' in col):
            continue
        if col not in t_run.colnames:
            continue
        t_run[col].unit = 'km/s'
    for col in Degparam:
        if col not in t_run.colnames:
            continue
        t_run[col].unit = 'deg'
    for col in Arcsparam:
        if col not in t_run.colnames:
            continue
        t_run[col].unit = 'arcsec'
    for col in Pixparam:
        t_run[col].unit = 'pixel'
    if 'offset' in t_run.colnames:
        t_run['offset'].description = 'offset from ned position. Only not equal to 0 when a 2D-Gaussian-fitted position is used'
    t_run.write('mergeTable/'+run+'.txt',overwrite=True,delimiter=',',format='ascii.ecsv')

reading ringlog2 for UGC00809 failed, probably because the table is empty
reading ringlog2 for UGC00809 failed, probably because the table is empty
['GAL_NAME', 'RACTR', 'DCCTR', 'offset', 'XPOSFIT(-1/0/1)', 'YPOSFIT(-1/0/1)', 'INCFIT(-1/0/1)', 'PAFIT(-1/0/1)', 'VSYSFIT(-1/0/1)', 'VDISPFIT(0/1)', 'VROTFIT(0/1)', 'FTYPE', 'WFUNC', 'BADFLAG', 'RAD', 'VROT', 'DISP', 'INC', 'P.A.', 'Z0', 'XPOS', 'YPOS', 'VSYS', 'E_VROT1', 'E_VROT2', 'E_DISP1', 'E_DISP2', 'E_PA1', 'E_PA2', 'E_VSYS1', 'E_VSYS2']
['GAL_NAME', 'RACTR', 'DCCTR', 'offset', 'XPOSFIT(-1/0/1)', 'YPOSFIT(-1/0/1)', 'INCFIT(-1/0/1)', 'PAFIT(-1/0/1)', 'VSYSFIT(-1/0/1)', 'VDISPFIT(0/1)', 'VROTFIT(0/1)', 'FTYPE', 'WFUNC', 'BADFLAG', 'RAD', 'VROT', 'DISP', 'INC', 'P.A.', 'Z0', 'XPOS', 'YPOS', 'VSYS', 'E_VROT1', 'E_VROT2', 'E_DISP1', 'E_DISP2', 'E_PA1', 'E_PA2', 'E_VSYS1', 'E_VSYS2']
reading ringlog2 for UGC00809 failed, probably because the table is empty
reading ringlog2 for UGC00809 failed, probably because the table is empty
['GAL_NAME

In [31]:
colnames_keep=['RADIUS','NPIX','SURFDENS','ERR_SD']
Densparam = ['SURFDENS','ERR_SD']
for run in runs:
    densprof_run=[]
    for gal in gallist:
        if not os.path.isfile(run+'/output/'+gal[:8]+'/'+'densprof'+'.txt'):
            continue
        try:
            gal_densprof = Table.read(run+'/output/'+gal[:8]+'/'+'densprof'+'.txt',format='ascii')
        except:
            print('reading densprof for '+gal+' failed, probably because the table is empty')
            continue
        colnames = 'RADIUS         SUM        MEAN      MEDIAN      STDDEV         MAD   NPIX     SURFDENS       ERR_SD  SURFDENS_FO   MSURFDENS  MSURFDENS2'.split()
        densprof_table = Table()
        densprof_table['GAL_NAME']=[gal]*len(gal_densprof)
        for colnumber in range(len(colnames)):
            if colnames[colnumber] in colnames_keep:
                densprof_table[colnames[colnumber]]=gal_densprof['col'+str(colnumber+1)]
        densprof_run.append(densprof_table)
        
    t_run_densprof = vstack(densprof_run)
    t_run_densprof['RADIUS'].unit = 'arcsec'
    t_run_densprof['NPIX'].description = '# of pixels in ring'
    for col in Densparam:
        t_run_densprof[col].unit = 'Jy*km/s/arcsec**2'
    t_run_densprof['SURFDENS'].description = 'average intensity in ring, not corrected for inclination'
    t_run_densprof['ERR_SD'].description = 'standard deviation of intensity in ring'
    
    t_run_densprof.write('mergeTable/'+run+'_densprof.txt',overwrite=True,delimiter=',',format='ascii.ecsv')

In [32]:
t = Table.read('mergeTable/gal_nrad8.txt',format='ascii.ecsv')